Import pacakges

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from utility import to_pickled_df



<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


### Before started
make sure to have the following files from kaggle retailrocket dataset and saved them in the folder called `data2`(feel free to change the folder but remeber to change the folder path in following code as well):
- `events.csv`
- `item_properties_part1.csv`
- `item_properties_part2.csv`

I did not use the orignial preprocess_kaggle.py since it would encode item ids. I want the orignal item id.

**create sorted data**

In [ ]:

data_directory = 'data2'
event_df = pd.read_csv(os.path.join(data_directory, 'events.csv'), header=0)
event_df.columns = ['timestamp','session_id','behavior','item_id','transid']
###remove transid column
event_df =event_df[event_df['transid'].isnull()]
event_df = event_df.drop('transid',axis=1)
##########remove users with <=2 interactions
event_df['valid_session'] = event_df.session_id.map(event_df.groupby('session_id')['item_id'].size() > 2)
event_df = event_df.loc[event_df.valid_session].drop('valid_session', axis=1)
##########remove items with <=2 interactions
event_df['valid_item'] = event_df.item_id.map(event_df.groupby('item_id')['session_id'].size() > 2)
event_df = event_df.loc[event_df.valid_item].drop('valid_item', axis=1)
######## transform to ids
# item_encoder = LabelEncoder()
session_encoder= LabelEncoder()
behavior_encoder=LabelEncoder()
# event_df['item_id'] = item_encoder.fit_transform(event_df.item_id)
event_df['session_id'] = session_encoder.fit_transform(event_df.session_id)
event_df['behavior']=behavior_encoder.fit_transform(event_df.behavior)
###########sorted by user and timestamp
event_df['is_buy']=1-event_df['behavior']
event_df = event_df.drop('behavior', axis=1)
sorted_events = event_df.sort_values(by=['session_id', 'timestamp'])



In [ ]:
sorted_events.head()

,timestamp,session_id,item_id,is_buy
1361687,1442004589439,0,285930,0
1367212,1442004759591,0,357564,0
1367342,1442004917175,0,67045,0
742616,1438969904567,1,325215,0
735273,1438970013790,1,325215,0


In [ ]:
# read item properties files
item_prop_1 = pd.read_csv("/Users/yayun/Desktop/MIDS/2023Spring/AIPI531/aipi_final_project/SA2C_code/Kaggle/data/item_properties_part1.csv")
item_prop_1.head()
item_prop_2 = pd.read_csv("/Users/yayun/Desktop/MIDS/2023Spring/AIPI531/aipi_final_project/SA2C_code/Kaggle/data/item_properties_part2.csv")
item_prop_2.head()

,timestamp,itemid,property,value
0,1433041200000,183478,561,769062
1,1439694000000,132256,976,n26.400 1135780
2,1435460400000,420307,921,1149317 1257525
3,1431831600000,403324,917,1204143
4,1435460400000,230701,521,769062


In [ ]:
# Concat 2 item properties files
item_prop = pd.concat([item_prop_1, item_prop_2])
item_prop.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


**Check if each item correspond to at least one category id**

In [ ]:
# check item id 
item_prop['itemid'].nunique() == item_prop[item_prop.property == 'categoryid']['itemid'].nunique()

True

In [ ]:
# check number of unique categories
item_prop[item_prop.property == 'categoryid']['value'].nunique()

1242

I only include item with category id, while this step might not be necessary.

In [ ]:
# only include item with categoryid
item_prop = item_prop[item_prop['property']=='categoryid']
# drop category id column 
item_prop = item_prop.drop('property', axis=1)
item_prop.head()

,timestamp,itemid,value
0,1435460400000,460429,1338
140,1432436400000,281245,1277
151,1435460400000,35575,1059
189,1437274800000,8313,1147
197,1437879600000,55102,47


In [ ]:
# sorted by itemid and time
item_prop = item_prop.sort_values(by=['itemid', 'timestamp'])
item_prop.head()

,timestamp,itemid,value
8220985,1431226800000,0,209
8428979,1431226800000,1,1114
9157980,1431226800000,2,1305
3828355,1433041200000,3,1171
8767464,1431831600000,4,1038


In [ ]:
# drop timestamp column
item_prop = item_prop.drop('timestamp', axis=1)
# reset index
item_prop = item_prop.reset_index(drop=True)

In [ ]:
# rename value as category id
item_prop = item_prop.rename(columns={'value': 'categoryid'})
item_prop.head()

,itemid,categoryid
0,0,209
1,1,1114
2,2,1305
3,3,1171
4,4,1038


In [ ]:
# # tranfrom the data to wide format
# item_prop_wide = item_prop.pivot(index=['itemid', 'timestamp'], columns='property', values='value')
# # make itemid and timestamp to columns
# item_prop_wide.reset_index(inplace=True)
# item_prop_wide.head()
# # create a new time column that can be merged with the property data
# sorted_events['time'] = sorted_events['timestamp'].apply(lambda x: int(str(x)[:8]+"00000"))
# sorted_events.head(3)


**merge sorted event data and corresponting property of the item**  
This step is not necessary. When we train, we used properties of all items instead of using property of a specific item so we do not need to know the property of that item. 

In [ ]:
# left merge on itemid and time
event_with_prop = pd.merge(sorted_events, item_prop, how='left', left_on=['item_id'], right_on=['itemid'])


In [ ]:
event_with_prop.isnull().sum()

timestamp         0
session_id        0
item_id           0
is_buy            0
itemid        49737
categoryid    49737
dtype: int64

In [ ]:
# drop null values
event_with_prop = event_with_prop.dropna()
# drop itemid 
event_with_prop = event_with_prop.drop('itemid', axis=1)
event_with_prop.head()

,timestamp,session_id,item_id,is_buy,categoryid
0,1442004589439,0,285930,0,1188
1,1442004759591,0,357564,0,256
2,1442004917175,0,67045,0,333
3,1438969904567,1,325215,0,299
4,1438970013790,1,325215,0,299


In [ ]:
# # one hot encdode categoryid
# event_with_prop = pd.get_dummies(event_with_prop, columns=['categoryid'])
# event_with_prop.head()

In [ ]:
# label encode categoryid and item_id
item_encoder = LabelEncoder()
event_with_prop['item_id'] = item_encoder.fit_transform(event_with_prop.item_id)
event_with_prop.head()

,timestamp,session_id,item_id,is_buy,categoryid
0,1442004589439,0,40321,0,1188
1,1442004759591,0,50422,0,256
2,1442004917175,0,9258,0,333
3,1438969904567,1,45801,0,299
4,1438970013790,1,45801,0,299


**make a dataframe with each item's corresponded one-hot-encodede category id.**

In [ ]:
# make a dataframe with each unique item id and one hot encoted category id
item_category = event_with_prop[['item_id', 'categoryid']].drop_duplicates()
item_category = pd.get_dummies(item_category, columns=['categoryid'])
# merge rows with same item_id
item_category = item_category.groupby('item_id').sum()
item_category.head()

,categoryid_0,categoryid_1,categoryid_100,categoryid_1000,categoryid_1001,categoryid_1002,categoryid_1003,categoryid_1006,categoryid_1007,categoryid_1008,...,categoryid_985,categoryid_987,categoryid_988,categoryid_99,categoryid_991,categoryid_992,categoryid_996,categoryid_997,categoryid_998,categoryid_999
item_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# save files
event_with_prop.to_csv('/Users/yayun/Desktop/MIDS/2023Spring/AIPI531/aipi_final_project/SA2C_code/Kaggle/data2/event_with_prop.csv', index=None, header=True)
event_with_prop.to_pickle('/Users/yayun/Desktop/MIDS/2023Spring/AIPI531/aipi_final_project/SA2C_code/Kaggle/data2/event_with_prop.pkl')
item_category.to_csv('/Users/yayun/Desktop/MIDS/2023Spring/AIPI531/aipi_final_project/SA2C_code/Kaggle/data2/item_category.csv', header=True)
item_category.to_pickle('/Users/yayun/Desktop/MIDS/2023Spring/AIPI531/aipi_final_project/SA2C_code/Kaggle/data2/item_category.pkl')

# Important
After saving these files, remeber to use `split_data.py` and `replay_buffer.py` to generate the following files:
- data_statis.pkl
- replay_buffer.pkl
- sampled_test.pkl
- sampled_train.pkl
- sampled_val.pkl